TODO: test batch size other than 1

# Preparation

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from IPython.display import clear_output
from sklearn.metrics import r2_score
from BinvoxDataset import CustomDataset

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: changli_824. Use `wandb login --relogin` to force relogin


True

# Create dataset

In [4]:
def transform(voxel):
    return torch.unsqueeze(torch.tensor(voxel, dtype = torch.float32), 0)

In [5]:
import json
paths = json.load(open('config.json', 'r'))
input_folder_path = paths['input_folder_path']
label_file_path = paths['label_file_path']

In [6]:
dataset = CustomDataset(input_folder_path = input_folder_path, label_file_path = label_file_path, transform = transform, max_count = None, ram_limit = 1000)
dataset_limited = CustomDataset(input_folder_path = input_folder_path, label_file_path = label_file_path, transform = transform, max_count = 5000, ram_limit = 1000)

# Define Network

In [7]:
class ConvNet(nn.Module):
    def __init__(self, kernel_size = 3, activation_fn = nn.ReLU()):
        super().__init__()

        self.max_pooling_2 = nn.MaxPool3d(kernel_size = 2)

        self.up_sampling_2 = nn.Upsample(scale_factor = 2)

        self.conv64_1_8 = nn.Sequential(
            nn.Conv3d(in_channels = 1, out_channels = 8, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 8),
            activation_fn
        )

        self.conv64_8_8 = nn.Sequential(
            nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 8),
            activation_fn
        )

        self.conv32_8_32 = nn.Sequential(
            nn.Conv3d(in_channels = 8, out_channels = 32, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 32),
            activation_fn
        )

        self.conv32_32_32 = nn.Sequential(
            nn.Conv3d(in_channels = 32, out_channels = 32, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 32),
            activation_fn
        )

        self.conv16_32_128 = nn.Sequential(
            nn.Conv3d(in_channels = 32, out_channels = 128, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 128),
            activation_fn
        )

        self.conv16_128_128 = nn.Sequential(
            nn.Conv3d(in_channels = 128, out_channels = 128, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 128),
            activation_fn
        )

        self.conv8_128_256 = nn.Sequential(
            nn.Conv3d(in_channels = 128, out_channels = 256, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 256),
            activation_fn
        )

        self.conv8_256_256 = nn.Sequential(
            nn.Conv3d(in_channels = 256, out_channels = 256, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 256),
            activation_fn
        )

        self.conv16_384_128 = nn.Sequential(
            nn.Conv3d(in_channels = 384, out_channels = 128, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 128),
            activation_fn
        )

        self.conv32_160_32 = nn.Sequential(
            nn.Conv3d(in_channels = 160, out_channels = 32, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 32),
            activation_fn
        )

        self.conv64_40_8 = nn.Sequential(
            nn.Conv3d(in_channels = 40, out_channels = 8, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 8),
            activation_fn
        )

        self.conv64_8_1 = nn.Sequential(
            nn.Conv3d(in_channels = 8, out_channels = 1, kernel_size = kernel_size, padding = 'same'),
            activation_fn
        )

    def forward(self, x):
        x = self.conv64_1_8(x)
        x = self.conv64_8_8(x)
        feature_map_64 = x.detach()
        x = self.max_pooling_2(x)
        x = self.conv32_8_32(x)
        x = self.conv32_32_32(x)
        feature_map_32 = x.detach()
        x = self.max_pooling_2(x)
        x = self.conv16_32_128(x)
        x = self.conv16_128_128(x)
        feature_map_16 = x.detach()
        x = self.max_pooling_2(x)
        x = self.conv8_128_256(x)
        x = self.conv8_256_256(x)
        x = self.up_sampling_2(x)
        x = torch.cat((feature_map_16, x), dim = 1)
        x = self.conv16_384_128(x)
        x = self.conv16_128_128(x)
        x = self.up_sampling_2(x)
        x = torch.cat((feature_map_32, x), dim = 1)
        x = self.conv32_160_32(x)
        x = self.conv32_32_32(x)
        x = self.up_sampling_2(x)
        x = torch.cat((feature_map_64, x), dim = 1)
        x = self.conv64_40_8(x)
        x = self.conv64_8_1(x)
        return x

In [8]:
class ConvNetScalarLabel(nn.Module):
    def __init__(self, kernel_size = 3, activation_fn = nn.ReLU()):
        super().__init__()

        self.max_pooling_2 = nn.MaxPool3d(kernel_size = 2)
        self.max_pooling_1 = nn.MaxPool3d(kernel_size = 1)
        self.max_pooling_8 = nn.MaxPool3d(kernel_size = 8)
        self.linear_1 = nn.Linear(256, 16)
        self.linear_2 = nn.Linear(16, 1)

        self.conv256_1_2 = nn.Sequential(
            nn.Conv3d(in_channels = 1, out_channels = 2, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 2),
            activation_fn
        )

        self.conv256_2_2 = nn.Sequential(
            nn.Conv3d(in_channels = 2, out_channels = 2, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 2),
            activation_fn
        )

        self.conv128_2_4 = nn.Sequential(
            nn.Conv3d(in_channels = 2, out_channels = 4, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 4),
            activation_fn
        )

        self.conv128_4_4 = nn.Sequential(
            nn.Conv3d(in_channels = 4, out_channels = 4, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 4),
            activation_fn
        )

        self.conv64_1_8 = nn.Sequential(
            nn.Conv3d(in_channels = 1, out_channels = 8, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 8),
            activation_fn
        )

        self.conv64_4_8 = nn.Sequential(
            nn.Conv3d(in_channels = 4, out_channels = 8, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 8),
            activation_fn
        )

        self.conv64_8_8 = nn.Sequential(
            nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 8),
            activation_fn
        )

        self.conv32_8_32 = nn.Sequential(
            nn.Conv3d(in_channels = 8, out_channels = 32, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 32),
            activation_fn
        )

        self.conv32_32_32 = nn.Sequential(
            nn.Conv3d(in_channels = 32, out_channels = 32, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 32),
            activation_fn
        )

        self.conv16_32_128 = nn.Sequential(
            nn.Conv3d(in_channels = 32, out_channels = 128, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 128),
            activation_fn
        )

        self.conv16_128_128 = nn.Sequential(
            nn.Conv3d(in_channels = 128, out_channels = 128, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 128),
            activation_fn
        )

        self.conv8_128_256 = nn.Sequential(
            nn.Conv3d(in_channels = 128, out_channels = 256, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 256),
            activation_fn
        )

        self.conv8_256_256 = nn.Sequential(
            nn.Conv3d(in_channels = 256, out_channels = 256, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = 256),
            activation_fn
        )

        self.conv4_256_32 = nn.Sequential(
            nn.Conv3d(in_channels = 256, out_channels = 32, kernel_size = 3, padding = 'same'),
            nn.BatchNorm3d(num_features = 32),
            activation_fn
        )

        self.conv4_32_32 = nn.Sequential(
            nn.Conv3d(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 'same'),
            nn.BatchNorm3d(num_features = 32),
            activation_fn
        )

        self.conv2_32_1 = nn.Sequential(
            nn.Conv3d(in_channels = 32, out_channels = 1, kernel_size = 1, padding = 'same'),
            nn.BatchNorm3d(num_features = 1),
            activation_fn
        )

    def forward(self, x):
        x = self.conv256_1_2(x)
        x = self.conv256_2_2(x)
        x = self.max_pooling_2(x)
        x = self.conv128_2_4(x)
        x = self.conv128_4_4(x)
        x = self.max_pooling_2(x)
        x = self.conv64_4_8(x)
        x = self.conv64_8_8(x)
        x = self.max_pooling_2(x)
        x = self.conv32_8_32(x)
        x = self.conv32_32_32(x)
        x = self.max_pooling_2(x)
        x = self.conv16_32_128(x)
        x = self.conv16_128_128(x)
        x = self.max_pooling_2(x)
        x = self.conv8_128_256(x)
        x = self.conv8_256_256(x)
        x = self.max_pooling_8(x)
        x = torch.squeeze(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        # print('outcome shape', torch.squeeze(x).shape)
        # return torch.reshape(x, (x.shape[0], 1))
        return torch.squeeze(x)

# Define Training Logic

In [9]:
def train_epoch(model, training_loader, optimizer, loss_fn):
    cumulative_loss = 0.0
    for i, data in enumerate(training_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = torch.squeeze(labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Make predictions
        outputs = model(inputs)

        # Compute loss and its gradients
        # print('label shape', labels.shape)
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        cumulative_loss += loss.item()
        
        wandb.log({'batch loss': loss.item()})
    return cumulative_loss / len(training_loader), cumulative_loss

In [10]:
def train(config, loss_fn):
    clear_output(wait = True)
    
    # initialize a wandb run
    wandb.init(config = config)

    # copy the config
    config = wandb.config
    
    print('config:', config)

    # get training loader
    training_loader = DataLoader(dataset, batch_size = config.batch_size, shuffle = False)

    # initialize model
    if config.activation_fn == 'ReLU':
        activation_fn = nn.ReLU()
    
    if config.activation_fn == 'Sigmoid':
        activation_fn = nn.Sigmoid()
    
    model = ConvNetScalarLabel(kernel_size = config.kernel_size, activation_fn = activation_fn).to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = config.learning_rate, momentum = 0.9)

    for epoch in range(config.epochs_choice):
        avg_loss_per_batch, cumulative_loss = train_epoch(model, training_loader, optimizer, loss_fn)
        wandb.log({'avg_loss_per_batch': avg_loss_per_batch, 'cumulative_loss': cumulative_loss})
        print(f'Loss for epoch {epoch}: {cumulative_loss}')
    
    return model

In [11]:
def test(config, model, loss_fn):
    # copy the config
    config = wandb.config
    
    # get testing loader
    testing_loader = DataLoader(dataset_limited, batch_size = config.batch_size, shuffle = False)
    
    testing_loss = 0.0
    y_true = []
    y_pred = []
    for i, data in enumerate(testing_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        testing_loss += loss.item()

        y_true.extend(labels.cpu().numpy().tolist())
        y_pred.extend(outputs.cpu().detach().numpy().tolist())
    return testing_loss / len(testing_loader), testing_loss, r2_score(y_true = y_true, y_pred = y_pred)

In [12]:
def evaluate(config = None):
    loss_fn = nn.L1Loss()
    model = train(config, loss_fn)
    avg_loss_per_batch_test, testing_loss, r2 = test(config, model, loss_fn)
    wandb.log({'avg_loss_per_batch_test': avg_loss_per_batch_test, 'testing_loss': testing_loss, 'r2': r2})

# Training settings

In [13]:
sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'testing_loss',
    'goal': 'minimize'
    }
sweep_config['metric'] = metric
parameters_dict = {
    'kernel_size': {
        'values': [3, 4, 5]
    },
    'activation_fn': {
        'values': ['ReLU', 'Sigmoid']
    },
    'epochs_choice': {
          'values': [5, 10, 20]
    },
    'learning_rate': {
        'values': [1e-4, 1e-3, 1e-2]
    },
    'batch_size': {
        'values': [8, 4]
    },
}

parameters_dict = {
    'kernel_size': {
        'values': [3]
    },
    'activation_fn': {
        'values': ['ReLU']
    },
    'epochs_choice': {
          'values': [5]
    },
    'learning_rate': {
        'values': [1e-3]
    },
    'batch_size': {
        'values': [4]
    },
}

sweep_config['parameters'] = parameters_dict

# Start

In [14]:
sweep_id = wandb.sweep(sweep_config, project = 'CNN_sweep_scalar')

Create sweep with ID: r3fuqrh4
Sweep URL: https://wandb.ai/changli_824/CNN_sweep_scalar/sweeps/r3fuqrh4


In [15]:
wandb.agent(sweep_id = sweep_id, function = evaluate)

config: {'activation_fn': 'ReLU', 'batch_size': 4, 'epochs_choice': 5, 'kernel_size': 3, 'learning_rate': 0.001}
Loading samples 0 through 999
Processing sample number 0
Loading samples 1000 through 1999
Processing sample number 1000
Loading samples 2000 through 2999
Processing sample number 2000
Loading samples 3000 through 3999
Processing sample number 3000
Loading samples 4000 through 4999
Processing sample number 4000
Loading samples 5000 through 5999
Processing sample number 5000
Loading samples 6000 through 6999
Processing sample number 6000
Loading samples 7000 through 7999
Processing sample number 7000
Loading samples 8000 through 8999
Processing sample number 8000
Loading samples 9000 through 9999
Processing sample number 9000
Loading samples 10000 through 10999
Processing sample number 10000
Loading samples 11000 through 11999
Processing sample number 11000
Loading samples 12000 through 12999
Processing sample number 12000
Loading samples 13000 through 13999
Processing sample

avg_loss_per_batch,█▄▂▂▁
avg_loss_per_batch_test,▁
batch loss,▁▁▂▁▁▁▁▁▂▁▁▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▂▁▁▁▁▁▂▁▁▁
cumulative_loss,█▄▂▂▁
r2,▁
testing_loss,▁
avg_loss_per_batch,3.73969
avg_loss_per_batch_test,3.12511
batch loss,0.26943
cumulative_loss,21245.20133
r2,-0.02955


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
